In [1]:
##PACKAGE IMPORTING
import pandas as pd
from dask.distributed import Client, progress ##For Progress Dashboard
import dask.dataframe as dd
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from ggplot import *
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time

C:\ProgramData\Anacondanew\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anacondanew\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

In [15]:
#SAVE YOUR MELTED DATAFRAME WITH CHUNCK OPTION
##ARGUMENT CHUNK SIZE IS DEFINED
%time sales_01 = pd.read_csv(r"C:\Users\Selvamani\Desktop\sales_train_validation.csv",chunksize=1000)
def Melt_Save(df,Melt_Vars_List):
    for chunk in tqdm.tqdm_notebook(sales_01):
        result = chunk.melt(id_vars= Melt_Vars_List)
        # export the results into a new file and append for each iteration using mode
        result.to_csv("C:/Users/Selvamani/Desktop/Master/main.csv", index=False , mode='a',header = False)
Melt_Save(sales_01,Melt_Vars_List)

Wall time: 4.98 ms


In [43]:
##PARALLEL PROCESSING STARTS HERE USING DASK
Column_Name_List = ['id','item_id','dept_id','cat_id','store_id','state_id' ,'variable' ,'value']
%time Raw_M_Data = dd.read_csv("C:/Users/Selvamani/Desktop/Master/main.csv",names=Column_Name_List,header=None)
%time print(Raw_M_Data.head())

Wall time: 186 ms
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id variable  value  
0       CA      d_1      0  
1       CA      d_1      0  
2       CA      d_1      0  
3       CA      d_1      0  
4       CA      d_1      0  
Wall time: 5 s


In [59]:
"Finding Top 10 Products Sold On an average per day"
%time Mean_Values = Raw_M_Data.groupby(['store_id']).value.mean().compute()

Wall time: 9min 22s


In [117]:
##WALMART LAST WEEK
df_sales_train=pd.read_csv(r"C:\Users\Selvamani\Desktop\sales_train_validation.csv")
cal=pd.read_csv(r"C:\Users\Selvamani\Desktop\Master\calendar.csv")
cal_train=pd.read_csv(r"C:\Users\Selvamani\Desktop\Master\cal_train.csv")
cal_pred = pd.read_csv(r"C:\Users\Selvamani\Desktop\Master\cal_pred.csv")
print(cal_train.columns)
cal_train=cal_train.drop(['Unnamed: 0', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year','d'],axis=1)
cal_pred=cal_pred.drop(['Unnamed: 0', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year','d'],axis=1)
print(type(cal_train))
cal_d = cal['d'].tolist()
cal_date = cal['date'].tolist()
cal_dict = dict(zip(cal_d, cal_date))
df1_sales_train=df_sales_train.iloc[:,:6]
df2_sales_train = df_sales_train.iloc[:,6:].rename(columns=cal_dict)
df_master=pd.concat([df1_sales_train,df2_sales_train],axis=1).drop('id',axis=1)
df_master.head()

Index(['Unnamed: 0', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'd', 'snap_CA', 'snap_TX', 'snap_WI', 'events_name_1_1',
       'events_name_2_1', 'events_type_1_1', 'events_type_2_1'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>


,item_id,dept_id,cat_id,store_id,state_id,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02,...,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [118]:
items=df_master['item_id'].unique()
state=df_master['state_id'].unique()
stores=df_master['store_id'].unique()
stores1={'CA_1':1,'CA_2':2,'CA_3':3,'CA_4':4,'TX_1':5,'TX_2':6,'TX_3':7,'WI_1':8,'WI_2':9,'WI_3':10}
state1={'CA':1,'TX':2,'WI':3}
fun=lambda x: 1 if (x == 5) | (x == 6) else (0)
fun1=lambda x:1 if x==True else(0)
df_comp=pd.DataFrame()
result=[]
j=1
start=time.time()
print(start)

1590428037.6249914


In [139]:
j=1
for i in items:
     if j<=1:
        print('entering for')
        print(i)
        df1=df_master.loc[df_master['item_id']== i ]
        df1=pd.melt(id_vars=['store_id'],value_vars=df_master.iloc[:,5:],var_name='Date',value_name='Sales',frame=df1)
        df1=df1.sort_values(['store_id','Date'])
        df1=pd.merge(df1, cal_train, how='left', left_on=['Date'], right_on=['date'])
        df1['dayofyear'] = pd.DatetimeIndex(df1['Date']).dayofyear
        df1['dayofweek'] = pd.DatetimeIndex(df1['Date']).dayofweek
        df1['weekofyear'] = pd.DatetimeIndex(df1['Date']).weekofyear
        df1['weekend'] = df1['dayofweek'].apply(fun)
        df1['monthstart'] = pd.DatetimeIndex(df1['Date']).is_month_start
        df1['monthstart'] = df1['monthstart'].apply(fun1)
        df1['quarterstart'] = pd.DatetimeIndex(df1['Date']).is_quarter_start
        df1['quarterstart'] = df1['quarterstart'].apply(fun1)
        df1['yearstart'] = pd.DatetimeIndex(df1['Date']).is_year_start
        df1['yearstart'] = df1['yearstart'].apply(fun1)
        df1['store_id']=df1['store_id'].map(stores1)
        df1 = df1.drop(['Date','date'], axis=1)
        df1_Sales = df1['Sales']
        df1 = df1.drop('Sales', axis=1)
        df1 = pd.concat([df1, df1_Sales], axis=1)
        predict_df = pd.DataFrame()
        K = 56
        elements = [1,2,3,4,5,6,7,8,9,10]
        predict_df['store_id'] = [ele for ele in elements for i in range(K)]
        predict_df['predict_dates'] = pd.concat([pd.DataFrame(pd.date_range(start='2016/04/25',end='2016/06/19'))]*10,axis=0,ignore_index=True)
        predict_df['dayofyear'] = pd.DatetimeIndex(predict_df['predict_dates']).dayofyear
        predict_df['dayofweek'] = pd.DatetimeIndex(predict_df['predict_dates']).dayofweek
        predict_df['weekofyear'] = pd.DatetimeIndex(predict_df['predict_dates']).weekofyear
        predict_df['weekend'] = predict_df['dayofweek'].apply(fun)
        predict_df['monthstart'] = pd.DatetimeIndex(predict_df['predict_dates']).is_month_start
        predict_df['monthstart'] = predict_df['monthstart'].apply(fun1)
        predict_df['quarterstart'] = pd.DatetimeIndex(predict_df['predict_dates']).is_quarter_start
        predict_df['quarterstart'] = predict_df['quarterstart'].apply(fun1)
        predict_df['yearstart'] = pd.DatetimeIndex(predict_df['predict_dates']).is_year_start
        predict_df['yearstart'] = predict_df['yearstart'].apply(fun1)
        cal_pred['date'] = pd.to_datetime(cal_pred['date'])
        Final_Prediction_File = pd.merge(cal_pred,predict_df, how='left', left_on=['date'], right_on=['predict_dates'])
        Final_Prediction_File = Final_Prediction_File.drop(['predict_dates','date'],axis=1)
        Final_Prediction_File = Final_Prediction_File.reindex(columns=['store_id', 'snap_CA', 'snap_TX', 'snap_WI', 'events_name_1_1',
       'events_name_2_1', 'events_type_1_1', 'events_type_2_1', 'dayofyear',
       'dayofweek', 'weekofyear', 'weekend', 'monthstart', 'quarterstart',
       'yearstart'])
        print(Final_Prediction_File.head())
        print(Final_Prediction_File.columns)
        print(df1.head())
        print(df1.columns)
        X=df1.drop('Sales',axis=1)
        y=df1['Sales']
        print("ENTERING MODEL")
        xgb=XGBRegressor()
        xgb.fit(X,y)
        y_pred=xgb.predict(Final_Prediction_File)
       # df_comp['y_test']=y_test
       # df_comp['y_pred']=y_pred
        #df_comp['y_pred']=df_comp['y_pred'].round()
       # rmse=np.sqrt(mean_squared_error(y_test,np.round(y_pred)))
       # result.append(rmse)
        print(y_pred)
        j=j+1        


entering for
HOBBIES_1_001
   store_id  snap_CA  snap_TX  snap_WI  events_name_1_1  events_name_2_1  \
0         1        0        0        0                0                0   
1         2        0        0        0                0                0   
2         3        0        0        0                0                0   
3         4        0        0        0                0                0   
4         5        0        0        0                0                0   

   events_type_1_1  events_type_2_1  dayofyear  dayofweek  weekofyear  \
0                0                0        116          0          17   
1                0                0        116          0          17   
2                0                0        116          0          17   
3                0                0        116          0          17   
4                0                0        116          0          17   

   weekend  monthstart  quarterstart  yearstart  
0        0           0     

C:\ProgramData\Anacondanew\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anacondanew\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[23:47:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[ 0.22542691  0.22575551  0.29786754  0.2635852   0.12427363  0.15884635
  0.15884635  0.05970696  0.05218282  0.08205882  0.22542691  0.22575551
  0.29786754  0.2635852   0.12427363  0.15884635  0.15884635  0.05970696
  0.05218282  0.08205882  0.21669713  0.21702573  0.28913775  0.25485545
  0.1237995   0.15837222  0.15837222  0.05923283  0.0517087   0.08158469
  0.19942027  0.19974887  0.2718609   0.24247018  0.1229409   0.15751362
  0.15751362  0.05837423  0.05085009  0.08072609  0.20002496  0.2087059
  0.28807366  0.24269944  0.13105345  0.16562617  0.16562617  0.08060643
  0.07308233  0.10295826  0.26911327  0.2777942   0.33844233  0.278472
  0.15542307  0.1899958   0.1899958   0.10464558  0.09712145  0.12699741
  0.24767098  0.2563519   0.33695582  0.2716257   0.14857677  0.18314949
  0.18314949  0.09779927  0.09027514

In [227]:
###PREDICTION FILE PROCESS
values = df1_sales_train['id'].unique()
#Unique_Values = pd.concat([values]*28,axis=0,ignore_index=True)
Unique_Values = [ele for ele in values for i in range(0,28)]
With_028 = pd.read_csv(r"C:\Users\Selvamani\Downloads\sub_0_28_day_xgb.csv")
With_2856 = pd.read_csv(r"C:\Users\Selvamani\Downloads\sub_28_56_day_xgb.csv")
With_028 = With_028.drop(['Unnamed: 0','index'],axis =1)
With_2856 = With_2856.drop(['Unnamed: 0','index'],axis =1)
With_028.head()
#With_028 = pd.concat([Unique_Values,With_028],axis=1)
#With_2856 = pd.concat([Unique_Values,With_2856],axis=1)
print(len(Unique_Values))
With_028['id'] = Unique_Values
print(With_028.head())
With_028.columns = ['store_id','Date','Sales','id']
#With_2856.columns = ['id','index','store_id','Date','Sales']
With_028 = With_028.drop(['store_id'],axis=1)
#With_2856 = With_2856.drop(['store_id'],axis=1)
#Main_df = pd.concat([With_028,With_2856],axis=0,ignore_index=True)
With_028.head()

853720
   store_id predict_dates         0                             id
0         1    2016-04-25  0.225427  HOBBIES_1_001_CA_1_validation
1         1    2016-04-25  0.108812  HOBBIES_1_001_CA_1_validation
2         1    2016-04-25  0.086473  HOBBIES_1_001_CA_1_validation
3         1    2016-04-25  1.388951  HOBBIES_1_001_CA_1_validation
4         1    2016-04-25  0.474618  HOBBIES_1_001_CA_1_validation


,Date,Sales,id
0,2016-04-25,0.225427,HOBBIES_1_001_CA_1_validation
1,2016-04-25,0.108812,HOBBIES_1_001_CA_1_validation
2,2016-04-25,0.086473,HOBBIES_1_001_CA_1_validation
3,2016-04-25,1.388951,HOBBIES_1_001_CA_1_validation
4,2016-04-25,0.474618,HOBBIES_1_001_CA_1_validation


In [208]:
Main_df = With_2856.pivot(index='id',columns='Date',values='Sales')
Final_Submission = pd.concat([Final_Submission_28,Final_Submission_56],axis=0,ignore_index=True)
Columns_Final_List = ['F{0}'.format(i) for i in range(0,29)]
Final_Submission.columns = Columns_Final_List
Submission_File = Submission_File.rename(columns={'F0':'id'})
sample_submission = pd.read_csv(r"C:\Users\Selvamani\Desktop\sample_submission.csv")
Submission_File['id'] = sample_submission['id']
Submission_File.tail()

ValueError: Index contains duplicate entries, cannot reshape

In [241]:
##FINAL TRY
Final_try = pd.read_csv(r"C:/Users/Selvamani/Downloads/pred_sub_xgb.csv")
values = df1_sales_train['id'].unique()
Unique_Values = [ele for ele in values for i in range(0,56)]
Final_try = Final_try.drop(['Unnamed: 0','store_id'],axis =1)
print(Final_try.head())
Final_try.columns=['Date','Sales']
print(len(Unique_Values))
Final_try['id'] = Unique_Values
print(Final_try.head())

  predict_dates         0
0    2016-04-25  0.225427
1    2016-04-26  0.225427
2    2016-04-27  0.216697
3    2016-04-28  0.199420
4    2016-04-29  0.200025
1707440
         Date     Sales                             id
0  2016-04-25  0.225427  HOBBIES_1_001_CA_1_validation
1  2016-04-26  0.225427  HOBBIES_1_001_CA_1_validation
2  2016-04-27  0.216697  HOBBIES_1_001_CA_1_validation
3  2016-04-28  0.199420  HOBBIES_1_001_CA_1_validation
4  2016-04-29  0.200025  HOBBIES_1_001_CA_1_validation


In [242]:
Final_try = Final_try.pivot(index='id',columns='Date')

In [243]:
Final_try.head()

Sales                                   \
Date                        2016-04-25 2016-04-26 2016-04-27 2016-04-28   
id                                                                        
FOODS_1_001_CA_1_validation   1.360472   1.354941   1.241045   1.027076   
FOODS_1_001_CA_2_validation   0.205979   0.197318   0.197318   0.202833   
FOODS_1_001_CA_3_validation   0.338196   0.321312   0.321312   0.321312   
FOODS_1_001_CA_4_validation   0.464410   0.471781   0.462314   0.472058   
FOODS_1_001_TX_1_validation   0.175486   0.168185   0.165246   0.183225   

                                                                         \
Date                        2016-04-29 2016-04-30 2016-05-01 2016-05-02   
id                                                                        
FOODS_1_001_CA_1_validation   1.106216   1.133749   1.191258   1.249626   
FOODS_1_001_CA_2_validation   0.203995   0.229893   0.248160   0.212600   
FOODS_1_001_CA_3_validation   0.411824   0.481349   0.458369   0.355666   
FOODS_1_001_CA_4_validation   0.513981   0.494423   0.562743   0.539272   
FOODS_1_001_TX_1_validation   0.193361   0.204941   0.178447   0.203990   

                                                   ...                        \
Date                        2016-05-03 2016-05-04  ... 2016-06-10 2016-06-11   
id                                                 ...                         
FOODS_1_001_CA_1_validation   1.255085   1.238775  ...   1.083038   1.179361   
FOODS_1_001_CA_2_validation   0.230456   0.206217  ...   0.201043   0.215095   
FOODS_1_001_CA_3_validation   0.277843   0.344411  ...   0.362046   0.623644   
FOODS_1_001_CA_4_validation   0.546643   0.531974  ...   0.536962   0.604398   
FOODS_1_001_TX_1_validation   0.188308   0.189334  ...   0.214678   0.209166   

                                                                         \
Date                        2016-06-12 2016-06-13 2016-06-14 2016-06-15   
id                                                                        
FOODS_1_001_CA_1_validation   1.241781   1.069492   1.057022   1.058641   
FOODS_1_001_CA_2_validation   0.215095   0.224249   0.176972   0.185875   
FOODS_1_001_CA_3_validation   0.637026   0.193380   0.230847   0.146660   
FOODS_1_001_CA_4_validation   0.599394   0.429158   0.441731   0.418155   
FOODS_1_001_TX_1_validation   0.203054   0.183724   0.169827   0.162527   

                                                                         
Date                        2016-06-16 2016-06-17 2016-06-18 2016-06-19  
id                                                                       
FOODS_1_001_CA_1_validation   1.041413   1.083038   1.174985   1.049144  
FOODS_1_001_CA_2_validation   0.199882   0.201043   0.226942   0.263750  
FOODS_1_001_CA_3_validation   0.237836   0.328348   0.681530   0.431763  
FOODS_1_001_CA_4_validation   0.422697   0.464620   0.579949   0.389778  
FOODS_1_001_TX_1_validation   0.191464   0.199574   0.209166   0.154712  

[5 rows x 56 columns]

In [244]:
sample_submission_01 = pd.DataFrame()
sample_submission_02 = pd.DataFrame()
sample_submission_01['id'] = Final_try.index
sample_submission_02['id'] = Final_try.index
Temp_01 = Final_try['Sales'].iloc[:,0:28]
Temp_02 = Final_try['Sales'].iloc[:,28:]
sample_submission_01 = pd.merge(left=sample_submission_01,right=Temp_01,on=['id'],how='right')
sample_submission_02 = pd.merge(left=sample_submission_02,right = Temp_02 ,on=['id'],how='right')
print(sample_submission_01.shape)
print(sample_submission_02.shape)

(30490, 29)
(30490, 29)


In [250]:
Columns_Final_List = ['F{0}'.format(i) for i in range(0,29)]
sample_submission_01.columns=Columns_Final_List
sample_submission_02.columns=Columns_Final_List
Submission_File = pd.concat([sample_submission_01,sample_submission_02],axis=0,ignore_index=True)
Submission_File = Submission_File.rename(columns={'F0':'id'})
sample_submission = pd.read_csv(r"C:\Users\Selvamani\Desktop\sample_submission.csv")
Submission_File['id'] = sample_submission['id']
cols = Submission_File.columns
Submission_File[cols[1:]] = Submission_File[cols[1:]].round(0)
Submission_File.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,...,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0
60976,FOODS_3_824_WI_3_evaluation,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
60977,FOODS_3_825_WI_3_evaluation,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
60978,FOODS_3_826_WI_3_evaluation,5.0,4.0,4.0,4.0,6.0,8.0,8.0,6.0,5.0,...,7.0,12.0,12.0,9.0,7.0,9.0,7.0,9.0,10.0,11.0
60979,FOODS_3_827_WI_3_evaluation,24.0,24.0,24.0,26.0,31.0,34.0,32.0,24.0,24.0,...,31.0,34.0,32.0,24.0,25.0,25.0,26.0,32.0,34.0,32.0


In [251]:
Submission_File.to_csv("C:/Users/Selvamani/Desktop/Master/sample_submission.csv",index=False)